<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access
This is a basic introduction on how to access EO data on a cloud provided using an Application Protocol Interface (API).

The main concepts that will presented here will use openEO as API, and the openEO Python Client library.

We will interact with cloud providers offering free and open data through STAC catalogs.

This functionality allows to create openEO workflows using the same syntax as when working connected to an openEO back-end, but works with local computing resources.

The main concepts presented in this notebook are:
- Lazy data loading
- Filter operators
- Reduce operators
- Apply operators
- Aggregate operators
- Create a simple EO workflow: load, filter, apply a function, extract information


In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

C:\Users\mclaus\Miniconda3\envs\sap\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


## Filter Operators

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators:

Let's start again with the same sample data from the Sentinel-2 STAC Collection:

In [2]:
url = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs"
spatial_extent = {"west": 11.1, "east": 11.5, "south": 46.1, "north": 46.5}

datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent)
datacube.execute()

C:\Users\mclaus\Documents\GitHub\openeo-python-client\openeo\metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")
Deserialised process graph into nested structure
Running process load_stac
kwargs: {'spatial_extent': <class 'openeo_pg_parser_networkx.pg_schema.BoundingBox'>, 'url': <class 'str'>}
--------------------------------------------------------------------------------


Walking node root-f78dcced-6a08-4695-bad0-d6f6149f1eec


<xarray.DataArray 'stackstac-e10464a23f36894405ae2216ed123873' (time: 751,
                                                                band: 17,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(751, 17, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/29)
  * time                          (time) datetime64[ns] 2016-11-05T10:12:57 ....
    id                            (time) <U24 'S2A_32TPS_20161105_0_L2A' ... ...
  * band                          (band) <U8 'overview' 'B11' ... 'visual' 'SCL'
  * x                             (x) float64 6.611e+05 6.612e+05 ... 6.932e+05
  * y                             (y) float64 5.153e+06 5.153e+06 ... 5.107e+06
    sentinel:utm_zone             int32 32
    ...                            ...
    sentinel:boa_offset_applied   (time) object None None None ... True True
    title                         (band) <U31 'True color image' ... 'Scene C...
    common_name                   (band) object None 'swir16' ... None None
    center_wavelength             (band) object None 1.6137 0.4439 ... None None
    full_width_half_max           (band) object None 0.143 0.027 ... None None
    epsg                          int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Temporal filter

To slice along time the required data collection with openEO, we can use the `filter_temporal` process.

In [3]:
temporal_extent = ['2022-06-10','2022-06-30']

temporal_slice = datacube.filter_temporal(temporal_extent)
temporal_slice.execute()

Deserialised process graph into nested structure
Running process load_stac
kwargs: {'spatial_extent': <class 'openeo_pg_parser_networkx.pg_schema.BoundingBox'>, 'url': <class 'str'>}
--------------------------------------------------------------------------------


Walking node root-57d482cb-887e-47f8-a3c7-139f4cc17644
Walking node loadstac1-57d482cb-887e-47f8-a3c7-139f4cc17644


Running process filter_temporal
kwargs: {'data': <class 'xarray.core.dataarray.DataArray'>, 'extent': <class 'openeo_pg_parser_networkx.pg_schema.TemporalInterval'>}
--------------------------------------------------------------------------------


<xarray.DataArray 'stackstac-f5b126b0a643ef78d5fd93dc20c4dfc0' (time: 8,
                                                                band: 17,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(8, 17, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/29)
  * time                          (time) datetime64[ns] 2022-06-10T10:28:04 ....
    id                            (time) <U24 'S2A_32TPS_20220610_0_L2A' ... ...
  * band                          (band) <U8 'overview' 'B11' ... 'visual' 'SCL'
  * x                             (x) float64 6.611e+05 6.612e+05 ... 6.932e+05
  * y                             (y) float64 5.153e+06 5.153e+06 ... 5.107e+06
    sentinel:utm_zone             int32 32
    ...                            ...
    sentinel:boa_offset_applied   (time) object True True True ... True True
    title                         (band) <U31 'True color image' ... 'Scene C...
    common_name                   (band) object None 'swir16' ... None None
    center_wavelength             (band) object None 1.6137 0.4439 ... None None
    full_width_half_max           (band) object None 0.143 0.027 ... None None
    epsg                          int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

### Spatial filter

In [4]:
## TODO: implementation missing in openeo-processes-dask -> PR here https://github.com/Open-EO/openeo-processes-dask/pull/118

In [5]:
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}

spatial_slice = datacube.filter_bbox(spatial_extent)
spatial_slice.execute()

Deserialised process graph into nested structure
Running process load_stac
kwargs: {'spatial_extent': <class 'openeo_pg_parser_networkx.pg_schema.BoundingBox'>, 'url': <class 'str'>}
--------------------------------------------------------------------------------


Walking node root-704cdbda-020b-40b0-b5db-1860b6467ee0
Walking node loadstac1-704cdbda-020b-40b0-b5db-1860b6467ee0


Running process filter_bbox
kwargs: {'data': <class 'xarray.core.dataarray.DataArray'>, 'extent': <class 'openeo_pg_parser_networkx.pg_schema.BoundingBox'>}
--------------------------------------------------------------------------------


<xarray.DataArray 'stackstac-1c8a5d28a7465231c4beb70b072b8299' (time: 751,
                                                                band: 17,
                                                                y: 489, x: 1145)>
dask.array<getitem, shape=(751, 17, 489, 1145), dtype=float64, chunksize=(1, 1, 382, 860), chunktype=numpy.ndarray>
Coordinates: (12/29)
  * time                          (time) datetime64[ns] 2016-11-05T10:12:57 ....
    id                            (time) <U24 'S2A_32TPS_20161105_0_L2A' ... ...
  * band                          (band) <U8 'overview' 'B11' ... 'visual' 'SCL'
  * x                             (x) float64 6.733e+05 6.733e+05 ... 6.848e+05
  * y                             (y) float64 5.153e+06 5.153e+06 ... 5.148e+06
    sentinel:utm_zone             int32 32
    ...                            ...
    sentinel:boa_offset_applied   (time) object None None None ... True True
    title                         (band) <U31 'True color image' ... 'Scene C...
    common_name                   (band) object None 'swir16' ... None None
    center_wavelength             (band) object None 1.6137 0.4439 ... None None
    full_width_half_max           (band) object None 0.143 0.027 ... None None
    epsg                          int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Bands filter

In [6]:
bands = ["B04","B08"]
# TODO: fix datacube metadata from stac
bands_slice = datacube.filter_bands(bands)
bands_slice.execute()

ValueError: Invalid band name/index 'B04'. Valid names: ['unknown']